In [34]:
import random
import numpy as np


from numpy import pi as π

import perceval as pv
from scipy.optimize import minimize
from perceval.components import *

from collections import Counter

In [ ]:
# Fraction of [1, 0, x, y] states
def getFidelityRho1(counts):
    totCounts, validCounts = 0, 0

    for state in counts:
        totCounts += counts[state]
        if state[0] == 1 and state[1] == 0:
            validCounts += counts[state]

    return validCounts/totCounts


# Fraction of [x, y, 1, 0] states
def getFidelityRho2(counts):
    totCounts, validCounts = 0, 0

    for state in counts:
        totCounts += counts[state]
        if state[2] == 1 and state[3] == 0:
            validCounts += counts[state]

    return validCounts/totCounts

def getFidelities(counts):
    return (getFidelityRho1(counts), getFidelityRho2(counts))

def costFunctionAtPhi(counts):
    return (1 - getFidelityRho1(counts))**2 + (1 - getFidelityRho2(counts))**2 + (getFidelityRho1(counts)-getFidelityRho2(counts))**2

In [ ]:
def loss_function(theta = [0] * 12):
    cost = 0
    Phi = [ 0, π/2, π, 3*π/2 ]

    # Evaluate the 4 cost functions, one for each fidelity
    for phi_i in Phi:
        circuit = getCircuit(evolTheta=theta, prepTheta=π/2, prepPhi=phi_i)
        p = pv.Processor("SLOS", circuit)
        p.with_input(pv.BasicState([0, 1, 0, 1]))

        # The sampler holds 'probs', 'sample_count' and 'samples' calls. You can use the one that fits your needs!
        sampler = pv.algorithm.Sampler(p)  
        sample_count = sampler.sample_count(1000)['results']

        # print(sample_count['results'])
        # sv = pv.StateVector([0,1,0,1])
        # circuit = getCircuit(evolTheta=theta, prepTheta=π/2, prepPhi=phi_i)

        # stepper = pv.simulators.Stepper(pv.backends.SLOSBackend())
        # stepper.set_circuit(circuit)

        

        # for r, c in circuit:
        #     sv = stepper.apply(sv, r, c)
        # for c in sample_count:
        #     print(c, sample_count[c])
        # counts = Counter()
        # for s in sv.samples(10000):
        #     if s[0] != s[1] and s[2] != s[3]: #Since we introduce only 2 photons in the circuit it is enough
        #         counts[s] += 1

        cost += costFunctionAtPhi(sample_count)
        
    return cost

In [61]:
def getFidelity(x0):
    phi = np.pi/2
    circuit = getCircuit(evolTheta=x0, prepTheta=π/2, prepPhi=phi)

    p = pv.Processor("SLOS", circuit)
    p.with_input(pv.BasicState([0, 1, 0, 1]))

    # The sampler holds 'probs', 'sample_count' and 'samples' calls. You can use the one that fits your needs!
    sampler = pv.algorithm.Sampler(p)  
    sample_count = sampler.sample_count(1000)['results']

    print(sample_count)
    print(getFidelityRho1(sample_count))
    print(getFidelityRho2(sample_count))

In [ ]:
loss_function()

3.2823080000000004

In [41]:
def optimCallback(intermediate_result):
    
    outLine = f'{intermediate_result.fun:0.3f}, '
    for x in intermediate_result.x:
        outLine += f'{x}, '
    outLine += '\n'

    print(outLine)
    with open('ResultFile.csv', 'a+') as outFile:
        outFile.writelines([outLine])

In [ ]:
class ao():
    def __init__(self, x0):
        self.x = x0

def minimizeVal(swapMimimizers = 3):
    initState = (np.random.rand(1, 12)[0])*2*π
    initState = [0]*12

    res = ao(initState)

    with open('ResultFile.csv', 'w') as outFile:
        outFile.writelines('Fun, p0, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11')

    for minimizerSwapIndex in range(swapMimimizers):

        res = minimize(loss_function,
            res.x,
            method="Nelder-Mead",
            bounds=[(0, 2*π)]*12,
            callback=optimCallback,
            options={''}
        )


        res = minimize(loss_function,
            res.x,
            method="powell",
            bounds=[(0, 2*π)]*12,
            callback=optimCallback
        )

        optimCallback(res)

    return res

In [ ]:
res = minimizeVal(3)

3.300, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

3.300, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

3.300, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

3.300, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

3.300, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

3.300, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

3.300, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

3.300, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

3.300, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

3.300, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

3.300, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

3.300, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

3.300, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

3.300, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

3.300, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 2.220184
       x: [ 5.716e+00  1.021e+00  3.868e+00  5.366e+00  2.410e+00
            5.347e+00  4.092e+00  1.072e+00  2.718e+00  5.128e+00
            2.400e+00  2.386e+00]
     nit: 5
   direc: [[ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00 ...  0.000e+00  0.000e+00]
           ...
           [ 0.000e+00  0.000e+00 ...  1.000e+00  0.000e+00]
           [ 0.000e+00  0.000e+00 ...  0.000e+00  1.000e+00]]
    nfev: 1220

In [62]:
getFidelity([5.716e+00,  1.021e+00,  3.868e+00,  5.366e+00,  2.410e+00,
5.347e+00,  4.092e+00,  1.072e+00,  2.718e+00,  5.128e+00,
2.400e+00,  2.386e+00])

{
  |2,0,0,0>: 5
  |1,1,0,0>: 1
  |1,0,1,0>: 231
  |1,0,0,1>: 90
  |0,2,0,0>: 9
  |0,1,1,0>: 502
  |0,1,0,1>: 157
  |0,0,2,0>: 3
  |0,0,0,2>: 2
}
0.321
0.733


In [ ]:
initialState = (np.random.rand(1, 12)[0])*2*π
initialState = [0]*12

with open('ResultFile.csv', 'w') as outFile:
    outFile.writelines('Fun, p0, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11')

res = minimize(loss_function,
    initialState,
    method="Nelder-Mead",
    bounds=[(0, 2*π)]*12,
    callback=optimCallback
)

optimCallback(res)

3.319, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00025, 0.0, 0.0, 0.0, 

3.319, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00025, 0.0, 0.0, 0.0, 

3.319, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00025, 0.0, 0.0, 0.0, 

3.294, 7.089120370370375e-06, 0.0, 7.089120370370375e-06, 7.8125e-06, 7.089120370370375e-06, 7.089120370370375e-06, 0.0, 7.089120370370375e-06, 7.089120370370375e-06, 7.089120370370375e-06, 7.089120370370375e-06, 7.089120370370375e-06, 

3.294, 7.089120370370375e-06, 0.0, 7.089120370370375e-06, 7.8125e-06, 7.089120370370375e-06, 7.089120370370375e-06, 0.0, 7.089120370370375e-06, 7.089120370370375e-06, 7.089120370370375e-06, 7.089120370370375e-06, 7.089120370370375e-06, 

3.294, 7.089120370370375e-06, 0.0, 7.089120370370375e-06, 7.8125e-06, 7.089120370370375e-06, 7.089120370370375e-06, 0.0, 7.089120370370375e-06, 7.089120370370375e-06, 7.089120370370375e-06, 7.089120370370375e-06, 7.089120370370375e-06, 

3.294, 7.089120370370375e-06, 0.0, 7.089120370370375e-06, 7.8125e-0

In [45]:
res.fun

3.252012

# VQC

In [44]:
# input = pv.BasicState([0, 1, 0, 1])
# backend = pv.SLOSBackend()

# circuit = getCircuit(prepTheta=π, prepPhi=π)

# param_circuit = circuit.get_parameters()
# params_init = [random.random()*π for _ in param_circuit]


# # Run the optimisation
# o = optimize.minimize(loss_function, params_init, method="Powell")

# print(f"The maximum probability is {-loss_function(o.x)}")

# # For n=4, the probability should be 3/32
# # The maximum can also be obtained with the Hadamard matrix :

# H4 = (1/2)*np.array([[1,1,1,1], [1,-1,1,-1], [1,1,-1,-1], [1,-1,-1,1]])
# backend.set_circuit(pv.Unitary(pv.Matrix(H4)))
# backend.set_input_state(input)
# backend.probability(output_to_max)

# pv.pdisplay(circuit)